In [1]:
import numpy as np
import re
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [2]:
def load_data():
    # lets load data
    txt_file = "vrp data.txt"
    data = []
    with open(txt_file, "r") as file:
        lines = file.readlines()  # Read all lines in the file
        column_parsed = False
        column_names = []
        if column_parsed == False:
            # Extract column names from first line
            words = re.findall("\S+", lines[0])
            for word in words:
                column_names.append(word)
            column_parsed = True

        for line in lines[1:]:  # Loop through lines, skipping the first line
            # Extract data points from line
            dataline = re.findall("\S+", line)
            line_dict = {}
            for i, point in enumerate(dataline):
                # Convert data point to float and store in dictionary
                line_dict[column_names[i]] = float(point)
            data.append(line_dict)  # Append dictionary to list of data
       


    #converting list of dict to numpy array
    data_array = np.array([[d[col] for col in column_names] for d in data])

    # #extract first row into depot array
    # DEPOT = data_array[0, :]

    # # extract rest of the rows into customers_array
    # customers_array = data_array[1:, :]
    # customers_array = add_distance_feature(customers_array,DEPOT)

    return data_array

In [3]:
customers_array = load_data()

In [4]:
def calculate_distance_matrix(customers_array):
    n_customers = len(customers_array)
    dist_matrix = np.zeros((n_customers, n_customers))

    # Loop over each pair of customers
    for i in range(n_customers):
        for j in range(n_customers):
            # Calculate the Euclidean distance between the XCOORD and YCOORD values of the two customers
            x_diff = customers_array[i][1] - customers_array[j][1]
            y_diff = customers_array[i][2] - customers_array[j][2]
            dist = np.sqrt(np.power(x_diff, 2) + np.power(y_diff, 2))

            # Store the calculated distance in the corresponding position in the distance matrix
            dist_matrix[i][j] = dist

    return dist_matrix


In [5]:

distance_matrix = calculate_distance_matrix(customers_array)
time_windows = customers_array[:,4:6].astype(int)
demands = customers_array[:,3:4].astype(int)
num_vehicles = 20 
vehicle_capacities = [200 for x in range(num_vehicles)] 
depot_index = 0 
time_limit_seconds = 3000 # time limit for calculation


In [6]:
def create_data_model(distance_matrix, time_windows, num_vehicles, demands, vehicle_capacities, depot_index):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = distance_matrix
    data['time_windows'] = time_windows.astype(int)
    data['num_vehicles'] = num_vehicles
    data['demands'] = demands
    data['vehicle_capacities'] = vehicle_capacities
    data['depot'] = depot_index
    return data
"""Capacitated Vehicles Routing Problem (CVRP) with Time Windows."""

# def print_solution(data, manager, routing, solution):
#     """Prints solution on console."""
#     total_distance = 0
#     total_load = 0
#     time_dimension = routing.GetDimensionOrDie('Time')
#     total_time = 0
    
#     for vehicle_id in range(data['num_vehicles']):
#         index = routing.Start(vehicle_id)
#         plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
#         route_load = 0
#         while not routing.IsEnd(index):
#             node_index = manager.IndexToNode(index)
#             time_var = time_dimension.CumulVar(index)
#             route_load += int(data['demands'][node_index])
#             plan_output += 'Place {0:>2} Arrive at {2:>2}min Depart at {3:>2}min (Load {1:>2})\n'.format(manager.IndexToNode(index), route_load, solution.Min(time_var), solution.Max(time_var))
            
#             previous_index = index
#             index = solution.Value(routing.NextVar(index))
            
            
#         time_var = time_dimension.CumulVar(index)
#         total_time += solution.Min(time_var)
#         plan_output +="Place {0:>2} Arrive at {2:>2}min \n\n".format(manager.IndexToNode(index), route_load, solution.Min(time_var), solution.Max(time_var))
        
#         # route output
#         plan_output += 'Load of the route: {}\n'.format(route_load)
#         plan_output += 'Time of the route: {}min\n'.format(solution.Min(time_var))
#         plan_output += "--------------------"
        
#         print(plan_output)
#         total_load += route_load

#     print('Total load of all routes: {}'.format(total_load))
#     print('Total time of all routes: {}min'.format(total_time))

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))


In [7]:
def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix, time_windows, num_vehicles, demands, vehicle_capacities, depot_index)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    
    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        10,  # allow waiting time
        3000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue

        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(int(time_window[0]), int(time_window[1]))

    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(int(data['time_windows'][0][0]),
                                                int(data['time_windows'][0][1]))

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = time_limit_seconds
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    return solution


In [8]:
solution = main()

Objective: 0
Route for vehicle 0:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 1:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 2:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 3:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 4:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 5:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 6:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 7:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 8:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 9:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 10:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 11:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 12:
0 Time(106,106) -> 3 Time(116,116) -> 0 Time(116,116)
Time of the route: 116min

Route 